In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite as bi
from scipy.sparse import linalg
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.linalg import fractional_matrix_power

def SIR_simulation(fname,transmission_prob, ini_h, ini_l, num_loop, infected_time, remove_edges=[]):
    print("extract_data")
    status_h, status_l, days = extract_data(fname,remove_edges)
    results = []
    peaks = []

    for i in range(num_loop):
        num_infected = []
#         print(status_h,status_l)
        status_h, status_l, time_h, time_l, at_least_one = initial_status(status_h, status_l, ini_h, ini_l)
#         print(sum(status_h.values()), sum(status_l.values()))
        for day in days:
            for visit in day:
                hcw, loc, duration = visit[5]+"w", visit[7]+"l", visit[4]
                if status_h[hcw]==1 and status_l[loc]==0:
                    if np.random.rand()< get_long_exposure_prob(int(duration), transmission_prob):
                        status_l[loc]=1
                        time_l[loc]=1
                        at_least_one.add(loc)
                elif status_h[hcw]==0 and status_l[loc]==1:
                    if np.random.rand()< get_long_exposure_prob(int(duration), transmission_prob):
                        status_h[hcw]=1
                        time_h[hcw]=1
                        at_least_one.add(hcw)
            status_h, status_l, time_h, time_l = update_status(status_h, status_l, time_h, time_l, infected_time)
            total_infected = list(status_h.values()).count(1) + list(status_l.values()).count(1)
            num_infected.append(total_infected)
        results.append(len(at_least_one)/(len(status_h)+len(status_l)))
        peaks.append(np.max(num_infected))
        for key in status_h.keys():
            status_h[key]=0
        for key in status_l.keys():
            status_l[key]=0
    return results,peaks

def extract_data(fname, remove_edges=[]):
#     fname = "outfile28.txt"
    infile = open(fname, "r")
    infection_status_h = {}
    infection_status_l = {}
    days = []
    visits = []
    remove_edges = set(remove_edges)
    next(infile)
    current = -1
    for line in infile:
        line = line.strip()
        line = line.split()
#         print(line)
        if line[0] not in days:
            current +=1
            days.append(line[0])
            visits.append([])
        try:
            infection_status_h[line[5]+"w"] = 0
            infection_status_l[line[7]+"l"] = 0
        except IndexError:
            print(line)
            return
        if (line[5]+"w",line[7]+"l") not in remove_edges:
            visits[current].append(line)
        print((line[5]+"w",line[7]+"l") in remove_edges)

    infile.close()
    return infection_status_h,infection_status_l,visits

def initial_status(hcw,loc,num_h, num_l):
    len_h = len(hcw)
    len_l = len(loc)
    time_h = {}
    time_l = {}
    print(len_h, len_l)
    infection_set = set()
    while sum(hcw.values()) < num_h and len(time_h) < len_h:
        key = list(hcw.keys())[np.random.randint(0,len_h)]
        infection_set.add(key)
        hcw[key] = 1
        time_h[key]=1
    while sum(loc.values()) < num_l and len(time_l) < len_l:
        key = list(loc.keys())[np.random.randint(0,len_l)]
        infection_set.add(key)
        loc[key]=1
        time_l[key]=1
#     print(infection_set)
    return hcw,loc, time_h, time_l, infection_set

def update_status(s_h, s_l, t_h, t_l, time):
    tl_key = list(t_l.keys())
    th_key = list(t_h.keys())
    for key in th_key:
        t_h[key]+=1
        if t_h[key]==time:
            t_h.pop(key)
            s_h[key] = 2
    for key in tl_key:
        t_l[key]+=1
        if t_l[key]==time:
            t_l.pop(key)
            s_l[key] = 2
    return s_h, s_l, t_h, t_l

def rand_edge(edges, alpha):

    ##edges = ([rows], [cols])
    choosen =set()
    len_e = len(edges[0])
    num = int(len_e*alpha)
    while len(choosen) < num:
        edge_idx = np.random.randint(0,len_e)
        choosen.add(edge_idx)
    return list(choosen)

def highest_weight(edges, weights, alpha):

    choosen =[]
    len_e = len(edges[0])
    num = int(len_e*alpha)
    print(len_e,num)
    args = np.argsort(-np.array(weights))[:num]

    return args

def mapping_edges(edges,edge_args,wor_map,loc_map):
    remove_edge_list=[]
    for i in edge_args:
#         print(edges[0][i],"aaaaa")
        remove_edge_list.append((wor_map[edges[0][i]],loc_map[edges[1][i]]))
    return remove_edge_list



def get_long_exposure_prob(visit_duration, trans_prob):

    long_exposure_trans_prob = 1 - np.math.pow((1 - trans_prob), np.math.ceil(visit_duration / 30)) # 30 seconds

    return long_exposure_trans_prob


def large_temporal_graph(fname, alphas, delta=0.2):
    print("=============================-")
    infile = open(fname, "r")
    days = []
    visits = []
    h_nodes={}
    l_nodes={}
    h_map = {}
    l_map = {}
    len_h=0
    len_l=0
    next(infile)
    current = -1
    for line in infile:
        line = line.strip()
        line = line.split()
#         print(line)
        if line[0] not in days:
            current +=1
            days.append(line[0])
            visits.append([])
        if line[5]+"w" not in h_nodes.keys():
            h_nodes[line[5]+"w"]=len_h
            h_map[len_h] = line[5]+"w"
            len_h+=1
        if line[7]+"l" not in l_nodes.keys():
            l_nodes[line[7]+"l"]=len_l
            l_map[len_l] = line[7]+"l"
            len_l+=1
        visits[current].append((line[5]+"w",line[7]+"l",get_long_exposure_prob(int(line[4]), delta)))

    mtx = csr_matrix((0,0))
    h_size = len(h_nodes)
    l_size = len(l_nodes)
#     print("======",h_size,l_size)
    day_size = len(visits)
    print(day_size, h_size,len_h, l_size,len_l)
    row = []
    col = []
    val = {}
    edge_set=[]
    edges = [[],[]]
    weight = []
    count = 0
    for day in visits:
        eye = sparse.identity(h_size+l_size)
        right_zero = csr_matrix((l_size,l_size))
        left_zero = csr_matrix((h_size,h_size))
        for visit in day:
            if (h_nodes[visit[0]],l_nodes[visit[1]]) not in val.keys():
                val[(h_nodes[visit[0]],l_nodes[visit[1]])] = (1-visit[2])
                if (h_nodes[visit[0]],l_nodes[visit[1]]) not in edge_set:
                    edge_set.append((h_nodes[visit[0]],l_nodes[visit[1]]))
                    edges[0].append(h_nodes[visit[0]])
                    edges[1].append(l_nodes[visit[1]])
            else:
                val[(h_nodes[visit[0]],l_nodes[visit[1]])] *= (1-visit[2])
        for key in val.keys():
            row.append(key[0])
            col.append(key[1])
            weight.append(1-val[key])
        B = csr_matrix((weight, (row, col)), shape=(h_size, l_size))
        B_t = B.transpose()
        right = sparse.vstack((B,right_zero))
        left = sparse.vstack((left_zero,B_t))
        A = sparse.hstack((left,right))
        if count!=day_size-1:
            A = sparse.hstack((A,eye))
            for i in range(count):
                zero = csr_matrix((l_size+h_size,l_size+h_size))
                A = sparse.hstack((zero,A))
            for j in range(day_size-count-2):
                zero = csr_matrix((l_size+h_size,l_size+h_size))
                A = sparse.hstack((A,zero))
        else:
            zero=csr_matrix((l_size+h_size,(l_size+h_size)*(day_size-1)))
            A = sparse.hstack((zero,A))

        mtx = sparse.vstack((mtx,A))
        col = []
        row = []
        val = {}
        weight = []
        count+=1

    x = linalg.eigs(mtx, k=1)
    print("eigs")

    centrality1 = []
    centrality2 = []
    centrality_a = 0
    centrality_b = 0
    centrality_dot = 0
    vec_a = []
    vec_b = []
    node_centrality_h={}
    node_centrality_l={}
    for i in range(h_size):
        vec_a.append([])
        for j in range(day_size):
            a = np.abs(x[1][i+(h_size+l_size)*j][0])
            vec_a[i].append(a)
            if j == 0:
                node_centrality_h[i] = a
            else:
                node_centrality_h[i]+= a

    for i in range(l_size):
        vec_b.append([])
        for j in range(day_size):
            b = np.abs(x[1][(i+h_size)+(h_size+l_size)*j][0])
            vec_b[i].append(b)
            if j == 0:
                node_centrality_l[i] = b
            else:
                node_centrality_l[i]+= b

    print("len_edge", len(edges[0]))

    for i in range(len(edges[0])):
        centrality1.append(node_centrality_h[edges[0][i]]*node_centrality_l[edges[1][i]])
        centrality2.append(np.dot(vec_a[edges[0][i]],vec_b[edges[1][i]]))

    alpha_dic = {}
    print(alphas)
    for alpha in alphas:
        num = int(len(edges[0])*alpha)
#         print(len(edges[0]), centrality1)
        summation = np.argsort(-np.array(centrality1))[:num]
        dot = np.argsort(-np.array(centrality2))[:num]
#         print(summation)
        s = []
        d = []


        for i in summation:
#             print("i",i, "h",h_map[edges[0][i]], "l", l_map[edges[1][i]])
            s.append((h_map[edges[0][i]],l_map[edges[1][i]]))

        for i in dot:
            d.append((h_map[edges[0][i]],l_map[edges[1][i]]))
        alpha_dic[alpha] = s,d

        print(len(s),len(d))
    return alpha_dic

def bipartite_graph(fname, delta=0.2):
#     fname = "outfile28.txt"
    infile = open(fname, "r")
    wor_set = {}
    loc_set = {}
    wor_map = {}
    loc_map = {}
    len_w = 0
    len_l = 0
    edge_dic = {}
    edge_list = []

    next(infile)
    for line in infile:
        line = line.strip()
        line = line.split()

        if line[5]+"w" not in wor_set.keys():
            wor_set[line[5]+"w"]=len_w
            wor_map[len_w] = line[5]+"w"
            len_w+=1
        if line[7]+"l" not in loc_set.keys():
            loc_set[line[7]+"l"] = len_l
            loc_map[len_l] = line[7]+"l"
            len_l+=1
        akey = (line[5]+"w",line[7]+"l")

        if akey in edge_dic.keys():
            edge_dic[akey]*= (1-get_long_exposure_prob(int(line[4]),delta))
        else:
            edge_dic[akey]=1-get_long_exposure_prob(int(line[4]),delta)
    infile.close()

    for key,val in edge_dic.items():
        lst = list(key)
        lst.append(1-val)
        tpl = tuple(lst)
        edge_list.append(tpl)
    G = nx.Graph()
    wor_list = np.array(list(wor_set.keys()))
    loc_list = np.array(list(loc_set.keys()))
    G.add_nodes_from(wor_list, bipartite=0)
    G.add_nodes_from(loc_list, bipartite=1)
    G.add_weighted_edges_from(edge_list)
    avg_clustering_coeff = bi.average_clustering(G)

    print(wor_list[:5],loc_list[:5])
    row=[]
    col=[]
    weights = []
    for key in edge_dic.keys():
            row.append(wor_set[key[0]])
            col.append(loc_set[key[1]])
            weights.append(1-edge_dic[key])
    # mean_degree = np.mean(weights)

    B = csr_matrix((weights, (row, col)), shape=(len_w, len_l))
    a = np.mean(B,axis=0)
    b = np.mean(B,axis=1)
    b = b.flatten()
    mean_degree = (np.mean(a[0])+np.mean(b[0]))/2
    nonzeros = B.nonzero()
    B_t = B.transpose()

    print("loc", len(loc_set),"wor", len(wor_set), B.shape)

    right_zero = csr_matrix((len(loc_set),len(loc_set)))
    left_zero = csr_matrix((len(wor_set),len(wor_set)))
    right = sparse.vstack((B,right_zero))
    left = sparse.vstack((left_zero,B_t))
    A = sparse.hstack((left,right))

    edges = (row,col)

    ### B is len(people) by len(location)
    ###A = [0 B]
#         [B'0]

    return (np.max(np.abs(linalg.eigs(A,k=1)[0])), avg_clustering_coeff, mean_degree, weights, edges,
           wor_map, loc_map)



In [ ]:
# graph_lst = [9, 10, 11]
graph_lst = [28]
alphas = [0.1, 0.3, 0.5, 0.7]
dic ={}

for graph in graph_lst:
    print(graph)
    fname = "outfile"+str(graph)+".txt"
    ei,c_coeff, deg, weights, edges, wor_map, loc_map = bipartite_graph(fname)
    normal = SIR_simulation(fname, 0.2, 10, 10, 20, 4)
    temporal = large_temporal_graph(fname,alphas)
    original_peak =np.mean(normal[1])
    original_total=np.mean(normal[0])
    peaks_h = []
    peaks_ran = []
    totals_h = []
    totals_ran = []
    totals_c1 = []
    totals_c2 = []
    peaks_c1 = []
    peaks_c2 = []
    for alpha in alphas:
        print(alpha)

        c1 = SIR_simulation(fname, 0.2, 10, 10, 20, 4, temporal[alpha][0])
        total_c1 = np.mean(c1[0])
        peak_c1 = np.mean(c1[1])

        c2 = SIR_simulation(fname, 0.2, 10, 10, 20, 4, temporal[alpha][1])
        total_c2 = np.mean(c2[0])
        peak_c2 = np.mean(c2[1])

        h_w = highest_weight(edges,weights,alpha)
        h_w_edges_removed = mapping_edges(edges, h_w, wor_map, loc_map)

        ran = rand_edge(edges,alpha)
        ran_edges_removed = mapping_edges(edges,ran, wor_map, loc_map)

        hw =SIR_simulation(fname, 0.2, 10, 10, 20, 4, h_w_edges_removed)
        print("hw finished")
        r = SIR_simulation(fname, 0.2, 10, 10, 20, 4, ran_edges_removed)
        print("ran finishied")
        total_h = np.mean(hw[0])
        peak_h = np.mean(hw[1])
        total_ran= np.mean(r[0])
        peak_ran = np.mean(r[1])

        peaks_h.append(peak_h/original_peak)
        peaks_ran.append(peak_ran/original_peak)
        totals_h.append(total_h/original_total)
        totals_ran.append(total_ran/original_total)
        totals_c1.append(total_c1/original_total)
        totals_c2.append(total_c2/original_total)
        peaks_c1.append(peak_c1/original_peak)
        peaks_c2.append(peak_c2/original_peak)
    dic[graph] = [peaks_h, peaks_ran, totals_h, totals_ran, peaks_c1, peaks_c2, totals_c1, totals_c2]



    name = "graph_"+str(graph)+ "_static_data.txt"
    outfile = open(name, "w")
    print("original_peak",original_peak, file=outfile)
    print("original_total",original_total, file=outfile)
    print("random_peak",peaks_ran, file=outfile)
    print("highest_weight_peak",peaks_h, file=outfile)
    print("random_total",totals_ran, file=outfile)
    print("highest_weight_total",totals_h, file=outfile)
    print("c1_total",totals_c1, file=outfile)
    print("c2_total",totals_c2, file=outfile)
    print("c1_peak",peaks_c1, file=outfile)
    print("c2_peak",peaks_c2, file=outfile)

    outfile.close()

print("save figs")

alpha = np.array(["0.1","0.3", "0.5", "0.7"])
index = np.array([0,1,2,3])
opacity=0.8
bar_width=0.1

for graph in graph_lst:
    fig, ax = plt.subplots()


    rects1 = plt.bar(index, dic[graph][1], bar_width,
    alpha=opacity,
    color='b',
    label='random')

    rects2 = plt.bar(index+bar_width, dic[graph][0], bar_width,
    alpha=opacity,
    color='g',
    label='highest weight')

    rects3 = plt.bar(index+bar_width*2, dic[graph][4], bar_width,
    alpha=opacity,
    color='r',
    label='centrality 1')

    rects4 = plt.bar(index+bar_width*3, dic[graph][5], bar_width,
    alpha=opacity,
    color='y',
    label='centrality 2')

    plt.xlabel('alpha')
    plt.ylabel('ratio')
    title = "Graph" + str(graph) + " Peak"
    plt.title(title)
    plt.xticks(index + bar_width, alpha)
    plt.legend()

    plt.tight_layout()
    plt.show()
    name = "graph_"+str(graph)+"_peak.png"
    fig.savefig(name)
    plt.close()

    fig, ax = plt.subplots()


    rects1 = plt.bar(index, dic[graph][3], bar_width,
    alpha=opacity,
    color='b',
    label='random')

    rects2 = plt.bar(index+bar_width, dic[graph][2], bar_width,
    alpha=opacity,
    color='g',
    label='highest weight')

    rects3 = plt.bar(index+bar_width*2, dic[graph][6], bar_width,
    alpha=opacity,
    color='r',
    label='centrality 1')

    rects4 = plt.bar(index+bar_width*3, dic[graph][7], bar_width,
    alpha=opacity,
    color='y',
    label='centrality 2')

    plt.xlabel('alpha')
    plt.ylabel('ratio')
    title = "Graph" + str(graph) + " Total"
    plt.title(title)
    plt.xticks(index + bar_width, alpha)
    plt.legend()

    plt.tight_layout()
    plt.show()
    name = "graph_"+str(graph)+"_total.png"
    fig.savefig(name)
    plt.close()


28
['1010133w' '1010151w' '1010229w' '1010145w' '1010119w'] ['1000012528l' '1000012521l' '1000012537l' '1000012527l' '1000012514l']
loc 58 wor 71 (71, 58)
extract_data
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
=============================-
73 71 71 58 58
eigs
len_edge 558
[0.1, 0.3, 0.5, 0.7]
55 55
167 167
279 279
390 390
0.1
extract_data
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
True
True
True
False
False
False
False
False
False
True
False
False
False
True
False
False
True
False
True
False
False
True
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
True
False
True
False
False
False
False
False
False
False
False
False
False
True
F

...

71 58
71 58
71 58
extract_data
True
False
False
True
False
False
False
False
False
True
True
False
True
True
True
True
True
True
False
True
False
True
False
True
False
False
False
False
True
False
False
False
False
True
True
True
True
False
True
True
False
True
False
True
False
False
False
False
False
True
False
False
True
True
True
True
False
False
False
False
False
False
True
False
False
False
True
False
False
True
False
True
False
False
True
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
True
False
True
False
False
False
False
False
False
False
False
False
False
True
False
True
True
False
False
False
False
False
True
False
False
True
True
False
False
True
True
True
True
False
True
True
False
False
False
False
False
False
False
False
False
True
False
False
True
True
True
False
False
False
T

False
True
False
False
False
True
False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
True
False
False
False
True
False
False
False
True
False
False
False
False
True
False
False
False
False
True
False
True
True
False
True
True
False
False
False
True
True
True
True
False
False
True
False
True
True
False
False
True
True
True
True
False
True
True
True
False
True
False
True
True
True
False
True
False
True
True
True
False
True
True
True
True
False
True
True
True
True
True
False
True
False
False
True
False
False
True
True
True
True
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fa

False
False
False
False
False
False
False
False
False
True
True
True
True
False
True
False
False
False
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
False
True
True
True
True
False
True
False
True
False
True
False
True
True
True
False
True
True
True
True
True
True
True
False
False
True
False
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
True
True
True
False
False
False
True
True
False
True
True
True
False
True
True
True
True
True
True
True
False
True
False
True
True
True
False
True
True
True
False
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
False
False
False
False
False
False
False
True
True
False
True
True
True
False
False
False
False
False
False
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
Tru

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
True
False
True
False
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
True
True
True
False
True
False
False
False
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Tru

71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
71 58
558 167
extract_data
True
False
False
True
False
False
False
False
False
True
True
False
True
True
True
True
True
True
False
True
False
True
False
True
False
False
False
False
True
False
False
False
False
True
True
True
True
False
False
True
False
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
False
True
False
True
True
True
True
True
True
True
True
False
True
True
False
True
False
True
False
True
False
True
True
True
True
True
False
True
False
True
True
True
False
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
True
True
False
True
False
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
False
True
False
True
False
True
False
True
False
True
True
True
True
False
True
Tru